# SQLAlchemy
> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

We're going to show how to create a database, add some data and do basic queries.
more complex queriex, doing migrations and database admin, are outside the scope of this lesson

## Design Questions & Concerns

1. Does it make sense to store Text and Target separately? 
    - Propose Record construct on the DB side to simplify (target is nullable).
        - A single text column for however many text inputs (separated by a split token like \<SPLIT\>, \<SEP\>, etc).
        - A single target column of type string (use type(eval(target)) to determine type, most likely int or list))
    - If accepted, we must combine label logger and text logger into a single record logger. 
2. Transformation provenance
    - Create transformation wrapper that only tracks arguments that differ from defaults
    - Do we store the transform name only or the full module heirarchy?
        - Full heirarchy (e.g. sibyl.transformations.text.contraction.expand_contractions.ExpandContractions)
    - How to determine transform_id at runtime? 
        - Query DB to see if it exists (likely to be speed inefficient)
    - Need to find a better way to serialize arguments because we cannot load via ast or json.

## Create a new database from scratch
Lets create a new database from scratch. we will
1. Create classes to define a schema
2. Map the scheme to a database
3. add objects to the database
4. run queries

> NOTE: we will use an in-memory database, but running with a file based one or a remote database would be just as easy

## Declare Models

In [2]:
from sqlalchemy import create_engine, select

import enum

from sqlalchemy import Table, Column, ForeignKey
from sqlalchemy import Integer, String, DateTime, TEXT, Enum, JSON
from sqlalchemy.orm import declarative_base, relationship, Session
from sqlalchemy.sql import func

from sqlalchemy_utils import database_exists

In [3]:
# enums

class RefGranularity(enum.Enum):
    character = 1
    word = 2
    sentence = 3
    paragraph = 4

In [4]:
Base = declarative_base()

# data tables
class Dataset(Base):

    __tablename__ = "Dataset"

    # columns
    id = Column(Integer, primary_key=True)
    name = Column(String)
    version = Column(String)
    # derived_from_dataset_id = Column(Integer, ForeignKey("Dataset.id"))
    # derived_from_dataset = relationship("Dataset", backref="derived_from_dataset_id")
    created_at = Column(DateTime(timezone=True), server_default=func.now())

class Record(Base):

    __tablename__ = "Record"

    # columns
    id = Column(Integer, primary_key=True)
    text = Column(TEXT, nullable=False)
    target = Column(String)
    dataset_id = Column(Integer, ForeignKey("Dataset.id"))
    created_at = Column(DateTime(timezone=True), server_default=func.now())

# transform tables
class Transform(Base):

    __tablename__ = "Transform"

    # columns
    id = Column(Integer, primary_key=True)
    module_name = Column(String)
    class_name = Column(String)
    trans_fn_name = Column(String)
    created_at = Column(DateTime(timezone=True), server_default=func.now())

class TransformApplied(Base):

    __tablename__ = "TransformApplied"

    # columns
    id = Column(Integer, primary_key=True)
    transformation_id = Column(Integer, ForeignKey("Transform.id"))
    transformation_class_args = Column(String)
    transformation_class_kwargs = Column(JSON)
    transformation_transform_args = Column(String)
    transformation_transform_kwargs = Column(JSON)
    input_record_id = Column(Integer, ForeignKey("Record.id"))
    output_record_id = Column(Integer, ForeignKey("Record.id"))
    diff = Column(JSON)
    diff_granularity = Column(Enum(RefGranularity))
    created_at = Column(DateTime(timezone=True), server_default=func.now())

# model tables
class ModelFramework(Base):

    __tablename__ = "ModelFramework"

    # columns
    id = Column(Integer, primary_key=True)
    name = Column(String)
    version = Column(String)
    created_at = Column(DateTime(timezone=True), server_default=func.now())

class Model(Base):

    __tablename__ = "Model"

    # columns
    id = Column(Integer, primary_key=True)
    name = Column(String)
    framework_id = Column(Integer, ForeignKey("ModelFramework.id"))
    version = Column(String)
    input_signature = Column(String)
    output_signature = Column(String)
    created_at = Column(DateTime(timezone=True), server_default=func.now())

class ModelInference(Base):

    __tablename__ = "ModelInference"

    # columns
    id = Column(Integer, primary_key=True)
    model_id = Column(Integer, ForeignKey("Model.id"))
    input_record_id = Column(Integer, ForeignKey("Record.id"))
    output = Column(String)
    created_at = Column(DateTime(timezone=True), server_default=func.now())

### Create an Engine

In [29]:
engine = create_engine("sqlite://", echo=True, future=True)

### Emit CREATE TABLE DDL

In [30]:
Base.metadata.create_all(engine)

2022-07-19 15:22:21,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-19 15:22:21,626 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Dataset")
2022-07-19 15:22:21,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-19 15:22:21,629 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Dataset")
2022-07-19 15:22:21,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-19 15:22:21,633 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Record")
2022-07-19 15:22:21,634 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-19 15:22:21,635 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Record")
2022-07-19 15:22:21,637 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-19 15:22:21,638 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Transform")
2022-07-19 15:22:21,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-19 15:22:21,641 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Transform")
2022-07-19 15:22:21,642 INFO sqlalchemy.engine.Engine [raw sql] 

### Create Objects and Persist

#### Transformations

In [31]:
from sibyl import TRANSFORMATIONS

In [135]:
def create_and_return(session, model, defaults=None, **kwargs):
    kwargs |= defaults or {}
    instance = model(**kwargs)
    try:
        session.add(instance)
        session.commit()
    except Exception:  
        session.rollback()
        instance = session.query(model).filter_by(**kwargs).one()
        return instance, False
    else:
        return instance, True

def create_and_return(session, model, defaults=None, **kwargs):
    kwargs |= defaults or {}
    instance = model(**kwargs)
    try:
        session.add(instance)
        session.commit()
    except Exception:  
        session.rollback()
        return instance, False
    else:
        return instance, True
        
def get_or_create(session, model, defaults=None, **kwargs):
    instance = session.query(model).filter_by(**kwargs).one_or_none()
    if instance:
        return instance, False
    else:
        kwargs |= defaults or {}
        instance = model(**kwargs)
        try:
            session.add(instance)
            session.commit()
        except Exception:  
            session.rollback()
            instance = session.query(model).filter_by(**kwargs).one()
            return instance, False
        else:
            return instance, True
        
def get_root_module(obj, sep="."):
    return obj.__module__.split(sep)[0]

In [136]:
with Session(engine) as session:
    for t in TRANSFORMATIONS:       
        t_instance, t_exists = get_or_create(
            session, 
            Transform,                        
            name=t.__name__)

NameError: name 'Session' is not defined

In [34]:
stmt = select(Transform)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-07-19 15:22:24,362 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-19 15:22:24,365 INFO sqlalchemy.engine.Engine SELECT "Transform".id, "Transform".name, "Transform".created_at 
FROM "Transform"
2022-07-19 15:22:24,367 INFO sqlalchemy.engine.Engine [generated in 0.00448s] ()
(1, 'ExpandContractions', datetime.datetime(2022, 7, 19, 22, 22, 23))
(2, 'ContractContractions', datetime.datetime(2022, 7, 19, 22, 22, 23))
(3, 'Emojify', datetime.datetime(2022, 7, 19, 22, 22, 23))
(4, 'AddPositiveEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(5, 'AddNegativeEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(6, 'AddNeutralEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(7, 'Demojify', datetime.datetime(2022, 7, 19, 22, 22, 23))
(8, 'RemovePositiveEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(9, 'RemoveNegativeEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(10, 'RemoveNeutralEmoji', datetime.datetime(2022, 7, 19, 22, 22, 23))
(11, 'ChangeLocation', datetime.dat

#### Text and Targets

In [11]:
in_text = [
    "The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil.", 
    "Unwatchable. You can't even make it past the first three minutes. And this is coming from a huge Adam Sandler fan!!1",
    "An unfunny, unworthy picture which is an undeserving end to Peter Sellers' career. It is a pity this movie was ever made.",
    "I think it's one of the greatest movies which are ever made, and I've seen many... The book is better, but it's still a very good movie!",
    "The only thing serious about this movie is the humor. Well worth the rental price. I'll bet you watch it twice. It's obvious that Sutherland enjoyed his role.",
    "Touching; Well directed autobiography of a talented young director/producer. A love story with Rabin's assassination in the background. Worth seeing"
]

in_target = [0, 0, 0, 1, 1, 1] # (imdb dataset 0=negative, 1=positive)

batch = (in_text, in_target)

In [12]:
with Session(engine) as session:
    
    records = [Record(text=text, target=target) for text, target in zip(*batch)]
    session.add_all(records)
    session.commit()

2022-07-19 14:00:23,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-19 14:00:23,430 INFO sqlalchemy.engine.Engine INSERT INTO "Record" (text, target, dataset_id) VALUES (?, ?, ?)
2022-07-19 14:00:23,431 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ("The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil.", 0, None)
2022-07-19 14:00:23,433 INFO sqlalchemy.engine.Engine INSERT INTO "Record" (text, target, dataset_id) VALUES (?, ?, ?)
2022-07-19 14:00:23,434 INFO sqlalchemy.engine.Engine [cached since 0.003994s ago] ("Unwatchable. You can't even make it past the first three minutes. And this is coming from a huge Adam Sandler fan!!1", 0, None)
2022-07-19 14:00:23,435 INFO sqlalchemy.engine.Engine INSERT INTO "Record" (text, target, dataset_id) VALUES (?, ?, ?)
2022-07-19 14:00:23,436 INFO sqlalchemy.engine.Engine [cached since 0.006554s ago] ("An unfunny, unworthy picture which is an undeserving end to Peter Sellers' c

In [13]:
stmt = select(Record)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-07-19 14:00:23,459 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-19 14:00:23,461 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".text, "Record".target, "Record".dataset_id, "Record".created_at 
FROM "Record"
2022-07-19 14:00:23,462 INFO sqlalchemy.engine.Engine [generated in 0.00312s] ()
(1, "The characters are unlikeable and the script is awful. It's a waste of the talents of Deneuve and Auteuil.", '0', None, datetime.datetime(2022, 7, 19, 21, 0, 23))
(2, "Unwatchable. You can't even make it past the first three minutes. And this is coming from a huge Adam Sandler fan!!1", '0', None, datetime.datetime(2022, 7, 19, 21, 0, 23))
(3, "An unfunny, unworthy picture which is an undeserving end to Peter Sellers' career. It is a pity this movie was ever made.", '0', None, datetime.datetime(2022, 7, 19, 21, 0, 23))
(4, "I think it's one of the greatest movies which are ever made, and I've seen many... The book is better, but it's still a very good movie!", '1', None, da

#### TransformApplied

In [5]:
%load_ext autoreload
%autoreload 2

In [7]:
from sibyl import *
from datasets import load_dataset

from lineage import LeBatch

from sqlalchemy import create_engine, select
from sqlalchemy_utils import database_exists

In [8]:
dataset = load_dataset("glue", "sst2", split="train[:1]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [9]:
scheduler = SibylTransformScheduler("sentiment")

In [18]:
text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []

batch_size = 1

scheduler.num_INV = 1
scheduler.num_SIB = 1

records = []
for i in range(0, len(label), batch_size):
    text_batch = text[i:i+batch_size]
    label_batch = label[i:i+batch_size]
    batch = (text_batch, label_batch)
    for transform in scheduler.sample():
        print(transform)
        # batch = transform.transform_batch(batch)
        batch = LeBatch(batch).apply(transform.transform_batch)
    records.append(batch)

In [11]:
batch

[<LeRecord:
 	 text="b'hide Modern secretion from the maternal unit  enshroud Modern secretion from the maternal unit '",
 	 target="[1.0, 0.0]",
 	 le_attrs={'transformation_provenance': <TransformationProvenance: {(1, '{"module_name": "sibyl.transformations.text.word_swap.change_synse", "class_name": "ChangeSynonym", "trans_fn_name": "transform_batch", "class_args": "null", "class_kwargs": "{\\"return_metadata\\": true}", "transform_args": "null", "transform_kwargs": "null", "fn_is_stochastic": false}'), (0, '{"module_name": "sibyl.transformations.text.mixture.text_mix", "class_name": "SentMix", "trans_fn_name": "transform_batch", "class_args": "null", "class_kwargs": "{\\"return_metadata\\": true}", "transform_args": "null", "transform_kwargs": "null", "fn_is_stochastic": false}')}>, 'prev': <LeRecord:
 	 text="b'hide new secretions from the parental units  hide new secretions from the parental units '",
 	 target="[1.0, 0.0]",
 	 le_attrs={'transformation_provenance': <Transformati

In [12]:
engine = create_engine("sqlite:///dpml.db", echo=True, future=True)
database_exists(engine.url)

True

In [19]:
stmt = select(TransformApplied)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-07-20 15:11:22,047 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 15:11:22,049 INFO sqlalchemy.engine.Engine SELECT "TransformApplied".id, "TransformApplied".transformation_id, "TransformApplied".transformation_class_args, "TransformApplied".transformation_class_kwargs, "TransformApplied".transformation_transform_args, "TransformApplied".transformation_transform_kwargs, "TransformApplied".input_record_id, "TransformApplied".output_record_id, "TransformApplied".diff, "TransformApplied".diff_granularity, "TransformApplied".created_at 
FROM "TransformApplied"
2022-07-20 15:11:22,050 INFO sqlalchemy.engine.Engine [cached since 502s ago] ()
(1, 1, 'null', '{"return_metadata": true}', 'null', 'null', 1, None, ['insert: [7,7]-[14,15]'], <RefGranularity.word: 2>, datetime.datetime(2022, 7, 20, 20, 23, 12))
(2, 2, 'null', '{"return_metadata": true}', 'null', 'null', None, None, ['replace: [12,14]-[12,14]'], <RefGranularity.word: 2>, datetime.datetime(2022, 7, 20, 20, 23, 13))
(3